In [1]:
import requests
import mysql.connector
import re
from multiprocessing import Process, Pool

In [2]:
db_cnx = mysql.connector.connect(user="docker", password="docker", host="localhost", database="tcc")

# Salvando todos os deputados das últimas tres legislaturas

In [10]:
listagem_deputados_url = "https://dadosabertos.camara.leg.br/api/v2/deputados?itens=100&idLegislatura=55,56,57"

In [11]:
def extrair_id_deputado(url):
    return re.search(r"/deputados/(.*)$", url).group(1)

In [12]:
## multi thread
hasNext = True
cursor = db_cnx.cursor()
statement = """
    insert ignore into deputado (id, nome, nome_eleitoral, situacao,
                          condicao_eleitoral, partido, estado, uri_foto)
    values ( %(id)s, %(nome)s, %(nome_eleitoral)s, %(situacao)s,
     %(condicao_eleitoral)s,
     %(partido)s,
     %(estado)s,
     %(uri_foto)s
     );
"""

def inserirParaleloAux(args):
    return inserirParalelo(*args)

def inserirParalelo(uri, params):
    detalhes = requests.get(uri).json()["dados"]
    params["nome"] = detalhes["nomeCivil"]
    params["situacao"] = detalhes["ultimoStatus"]["situacao"]
    params["condicao_eleitoral"] = detalhes["ultimoStatus"]["condicaoEleitoral"]
    return params

while hasNext:
    print(listagem_deputados_url)
    response = requests.get(listagem_deputados_url).json()
    print(len(response["dados"]))
    paralel_params = []
    for deputado in response["dados"]:
        params = {
            "id": deputado["id"],
            "nome_eleitoral": deputado["nome"],
            "estado": deputado["siglaUf"],
            "partido": deputado["siglaPartido"],
            "uri_foto": deputado.get("urlFoto")
        }
        paralel_params.append((deputado["uri"], params))
        
    with Pool(processes = 5) as pool:
        values = pool.map(inserirParaleloAux, paralel_params)
    print(f"processados = {len(values)}")
    cursor.executemany(statement, values)
    db_cnx.commit()
    hasNext = False
    
    search = re.search(r"pagina=([0-9]+)", listagem_deputados_url)
    if search is None:
        pagina_atual = 0
    else:
        pagina_atual = search.group(1)
    print(f"Pagina {pagina_atual}")
    for i in response["links"]:
        if i.get("rel") == "next":
            hasNext = True
            listagem_deputados_url = i["href"]
            break

https://dadosabertos.camara.leg.br/api/v2/deputados?itens=100&idLegislatura=55,56,57
100
processados = 100
Pagina 0
https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=55,56,57&pagina=2&itens=100
100
processados = 100
Pagina 2
https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=55,56,57&pagina=3&itens=100
100
processados = 100
Pagina 3
https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=55,56,57&pagina=4&itens=100
100
processados = 100
Pagina 4
https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=55,56,57&pagina=5&itens=100
100
processados = 100
Pagina 5
https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=55,56,57&pagina=6&itens=100
100
processados = 100
Pagina 6
https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=55,56,57&pagina=7&itens=100
100
processados = 100
Pagina 7
https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=55,56,57&pagina=8&itens=100
100
processados = 100
Pagina 8
https://d

In [ ]:
cursor.close()

In [ ]:
# ## single thread
# hasNext = True
# cursor = db_cnx.cursor()
# statement = """
#     insert ignore into deputado (id, nome, nome_eleitoral, situacao,
#                           condicao_eleitoral, partido, estado, uri_foto)
#     values ( %(id)s, %(nome)s, %(nome_eleitoral)s, %(situacao)s,
#      %(condicao_eleitoral)s,
#      %(partido)s,
#      %(estado)s,
#      %(uri_foto)s
#      );
# """
# 
# values = []
# progresso = 0
# while hasNext:
#     response = requests.get(listagem_deputados_url).json()
#     for deputado in response["dados"]:
#         cursor.execute("select exists(select * from deputado where id = %s)", [deputado["id"]])
#         deputado_existe = cursor.fetchone()[0] == 1
#         if deputado_existe:
#             continue
# 
#         params = {
#             "id": deputado["id"],
#             "nome_eleitoral": deputado["nome"],
#             "estado": deputado["siglaUf"],
#             "partido": deputado["siglaPartido"],
#             "uri_foto": deputado.get("urlFoto")
#         }
# 
#         detalhes = requests.get(deputado["uri"]).json()["dados"]
#         params["nome"] = detalhes["nomeCivil"]
#         params["situacao"] = detalhes["ultimoStatus"]["situacao"]
#         params["condicao_eleitoral"] = detalhes["ultimoStatus"]["condicaoEleitoral"]
# 
#         values.append(params)
#         progresso += 1
#     hasNext = False
#     if progresso % 100 == 0:
#         cursor.executemany(statement, values)
#         db_cnx.commit()
#         values = []
#         print(progresso)
#         search = re.search(r"pagina=([0-9]+)", listagem_deputados_url)
#         if search is None:
#             pagina_atual = 0
#         else:
#             pagina_atual = search.group(1)
#         print(f"Pagina {pagina_atual}")
#     for i in response["links"]:
#         if i.get("rel") == "next":
#             hasNext = True
#             listagem_deputados_url = i["href"]
#             break

In [ ]:
progresso

In [ ]:
db_cnx.close()
cursor.close()